In [7]:
!pip install pypdf
!pip install pdfplumber
!pip install PyPDF4

  Obtaining dependency information for pdfplumber from https://files.pythonhosted.org/packages/c1/f9/1a41afffe5e7a98ab9b6a6dd3dab9d99b677fae2536f676397c4506f6554/pdfplumber-0.10.2-py3-none-any.whl.metadata
  Obtaining dependency information for pypdfium2>=4.18.0 from https://files.pythonhosted.org/packages/3b/28/a62cc4c717be9441a987476e3b91028f989584e14ba6f7bf0892b3a28edc/pypdfium2-4.18.0-py3-none-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 5.1 MB/s eta 0:00:00
Using cached pypdfium2-4.18.0-py3-none-macosx_11_0_arm64.whl (2.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 1.3 MB/s eta 0:00:00 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for PyPDF4: filename=PyPDF4-1.27.0-py3-none-any.whl size=61225 sha256=4d1bf5b6e640bcd074bfcd

In [2]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("../src/data/sample.pdf")
pages = loader.load_and_split()

In [4]:
len(pages)

3

In [6]:
pages[2]

Document(page_content="●\nC o a c h i n g\na n d\nt r a i n i n g\nf o r\nt e a m\nm e m b e r\n●\nA p p l y\nD e s i g n\np a t t e r n\na n d\ns o f t w a r e\na r c h i t e c t u r e\nr i g o r o u s l y\nf o r\nC o m p a n y ' s\nC o r e\nF r a m e w o r k\n●\nR e s e a r c h\nD e e p\nL e a r n i n g\nt e c h n i q u e\nf o r\nm e d i c a l\ni m a g e\nc l a s s i ﬁ c a t i o n .\nResult:\n●\nC o n t r i b u t e d\ns u c c e s s f u l l y\nt o\nt h e\na r c h i t e c t u r e\nd e s i g n\na n d\nc o d i n g\np h a s e\no f\nc o m p a n y\nf r a m e w o r k\na s\nw e l l\na s\nn e w\np r o d u c t\nd e v e l o p m e n t\n●\nR e c e i v e d\ne x c e l l e n t\np e r f o r m a n c e\nr e v i e w\ne v e r y\ny e a r\nCodix,\nV i e t n a m\n—\nC\n&\nS Q L\nD e v e l o p e r \nh t t p s : / / w w w . c o d i x . e u / e n\nJ u l y\n2 0 1 3\n-\nF e b\n2 0 1 4\nD e v e l o p\nE n t e r p r i s e\nS o l u t i o n\nS o f t w a r e\nf o r\nF i n a n c i a l\nE u r o p e a n\nO r g a n i z a 

In [9]:
import pdfplumber
import PyPDF4
import re
import os
import sys
from typing import Callable, List, Tuple, Dict

from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

import time


from dotenv import load_dotenv

In [24]:
def extract_metadata_from_pdf(file_path: str) -> dict:
    with open(file_path, "rb") as pdf_file:
        reader = PyPDF4.PdfFileReader(pdf_file)  # Change this line
        metadata = reader.getDocumentInfo()
        return {
            "title": metadata.get("/Title", "").strip(),
            "author": metadata.get("/Author", "").strip(),
            "creation_date": metadata.get("/CreationDate", "").strip(),
        }


def extract_pages_from_pdf(file_path: str) -> List[Tuple[int, str]]:
    """
    Extracts the text from each page of the PDF.

    :param file_path: The path to the PDF file.
    :return: A list of tuples containing the page number and the extracted text.
    """
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")

    with pdfplumber.open(file_path) as pdf:
        pages = []
        for page_num, page in enumerate(pdf.pages):
            text = page.extract_text()
            if text.strip():  # Check if extracted text is not empty
                pages.append((page_num + 1, text))
    return pages




def parse_pdf(file_path: str) -> Tuple[List[Tuple[int, str]], Dict[str, str]]:
    """
    Extracts the title and text from each page of the PDF.

    :param file_path: The path to the PDF file.
    :return: A tuple containing the title and a list of tuples with page numbers and extracted text.
    """
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")

    metadata = extract_metadata_from_pdf(file_path)
    pages = extract_pages_from_pdf(file_path)

    return pages, metadata


def merge_hyphenated_words(text: str) -> str:
    return re.sub(r"(\w)-\n(\w)", r"\1\2", text)


def fix_newlines(text: str) -> str:
    return re.sub(r"(?<!\n)\n(?!\n)", " ", text)


def remove_multiple_newlines(text: str) -> str:
    return re.sub(r"\n{2,}", "\n", text)


def clean_text(
    pages: List[Tuple[int, str]], cleaning_functions: List[Callable[[str], str]]
) -> List[Tuple[int, str]]:
    cleaned_pages = []
    for page_num, text in pages:
        for cleaning_function in cleaning_functions:
            text = cleaning_function(text)
        cleaned_pages.append((page_num, text))
    return cleaned_pages



def text_to_docs(text: List[str], metadata: Dict[str, str]) -> List[Document]:
    """Converts list of strings to a list of Documents with metadata."""
    doc_chunks = []

    for page_num, page in text:
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
            chunk_overlap=200,
        )
        chunks = text_splitter.split_text(page)
        for i, chunk in enumerate(chunks):
            doc = Document(
                page_content=chunk,
                metadata={
                    "page_number": page_num,
                    "chunk": i,
                    "source": f"p{page_num}-{i}",
                    **metadata,
                },
            )
            doc_chunks.append(doc)

    return doc_chunks

#Without pages
def parse_pdf_without_pages(file_path: str) -> Tuple[List[str], Dict[str, str]]:
    """
    Extracts the title and text from each page of the PDF.

    :param file_path: The path to the PDF file.
    :return: A tuple containing the title and a list of tuples with page numbers and extracted text.
    """
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")

    metadata = extract_metadata_from_pdf(file_path)
    pages = extract_text_from_pdf(file_path)

    return pages, metadata

# Without pages
def extract_text_from_pdf(file_path: str) -> List[str]:
    """
    Extracts the text from each page of the PDF.

    :param file_path: The path to the PDF file.
    :return: A list of tuples containing the page number and the extracted text.
    """
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")

    with pdfplumber.open(file_path) as pdf:
        pages = []
        text = ""
        for page_num, page in enumerate(pdf.pages):
            text += page.extract_text()
        if text.strip():  # Check if extracted text is not empty
            pages.append(text)
    return pages

# Without pages
def clean_text_without_pages(
    pages: List[str], cleaning_functions: List[Callable[[str], str]]
) -> List[str]:
    cleaned_pages = []
    for text in pages:
        for cleaning_function in cleaning_functions:
            text = cleaning_function(text)
        cleaned_pages.append(text)
    return cleaned_pages

# Without pages
def text_to_docs_without_pages(text: List[str], metadata: Dict[str, str]) -> List[Document]:
    """Converts list of strings to a list of Documents with metadata."""
    doc_chunks = []

    for page in text:
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
            chunk_overlap=200,
        )
        chunks = text_splitter.split_text(page)
        for i, chunk in enumerate(chunks):
            doc = Document(
                page_content=chunk,
                metadata={
                    "chunk": i,
                    **metadata,
                },
            )
            doc_chunks.append(doc)

    return doc_chunks

In [11]:
if __name__ == "__main__":
    load_dotenv()

    start_time = time.time()

    # Step 1: Parse PDF
    file_path = "../src/data/sample.pdf"

    # convert(file_path)

    # if is_doc_or_docx_by_extension(file_path):
    #     convert(file_path)
    #     file_path = file_path.replace(".docx", ".pdf")

    raw_pages, metadata = parse_pdf(file_path)
    print(raw_pages)
    print(metadata)


    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time} seconds")

[(1, "District7\nDo Ngoc Tan\nHoChiMinhCity,Vietnam\n(+84)772954324\ndarkknightkhtn2008@gmail.com\nI’maSoftwareEngineerwhohas strongpassionwithinnovation\nscienceandtechnologies\nhttps://www.linkedin.com/in/do-ngoc-tan-64260072/\nEXPERIENCE SKILLS\nLanguages:C++20,Rust\nRustOpenSourceContributor-ForFun: Fluvioisa\nGolang,Python,Javascript\nhigh-performancedistributeddatastreamingplatformthat'swritteninRust,\nbuilttomakeiteasytorunreal-timeapplications-https://www.fluvio.io/\nMachineLearning:\nPytorch\nMay2023-Now\n● Contributetofluvioopensource: Blockchain:Tendermint,\nhttps://github.com/infinyon/fluvio Cosmos,Ethereum.\nAWARDS\nAStartupinDataPrivacy—LeadEngineer(Rust,C++)\nApril2023-Now TopWinnerofBlockchain\nHackathonEventHostedBy\nTake theinitiativetore-writesystemfromJStoRust\nPolicyPal\nWorldQuant,Vietnam—SeniorEngineer/Lead\nEngineer(C++):WorldQuant,LLCisaninternationalhedgefundand\nquantitativeinvestmentmanagementfirmheadquarteredinOldGreenwich, LANGUAGES\nConnecticut\nhttps://w

In [12]:
raw_pages

[(1,
  "District7\nDo Ngoc Tan\nHoChiMinhCity,Vietnam\n(+84)772954324\ndarkknightkhtn2008@gmail.com\nI’maSoftwareEngineerwhohas strongpassionwithinnovation\nscienceandtechnologies\nhttps://www.linkedin.com/in/do-ngoc-tan-64260072/\nEXPERIENCE SKILLS\nLanguages:C++20,Rust\nRustOpenSourceContributor-ForFun: Fluvioisa\nGolang,Python,Javascript\nhigh-performancedistributeddatastreamingplatformthat'swritteninRust,\nbuilttomakeiteasytorunreal-timeapplications-https://www.fluvio.io/\nMachineLearning:\nPytorch\nMay2023-Now\n● Contributetofluvioopensource: Blockchain:Tendermint,\nhttps://github.com/infinyon/fluvio Cosmos,Ethereum.\nAWARDS\nAStartupinDataPrivacy—LeadEngineer(Rust,C++)\nApril2023-Now TopWinnerofBlockchain\nHackathonEventHostedBy\nTake theinitiativetore-writesystemfromJStoRust\nPolicyPal\nWorldQuant,Vietnam—SeniorEngineer/Lead\nEngineer(C++):WorldQuant,LLCisaninternationalhedgefundand\nquantitativeinvestmentmanagementfirmheadquarteredinOldGreenwich, LANGUAGES\nConnecticut\nhttps:/

In [13]:
    # Step 2: Create text chunks
cleaning_functions = [
    merge_hyphenated_words,
    fix_newlines,
    remove_multiple_newlines,
    ]
cleaned_text_pdf = clean_text(raw_pages, cleaning_functions)
document_chunks = text_to_docs(cleaned_text_pdf, metadata)

#     # Optional: Reduce embedding cost by only using the first 23 pages
#     document_chunks = document_chunks[:70]

#     # Step 3 + 4: Generate embeddings and store them in DB
#     embeddings = OpenAIEmbeddings()
#     vector_store = Chroma.from_documents(
#         document_chunks,
#         embeddings,
#         collection_name="april-2023-economic",
#         persist_directory="src/data/chroma",
#     )

#     # Save DB locally
#     vector_store.persist()

In [14]:
cleaned_text_pdf

[(1,
  "District7 Do Ngoc Tan HoChiMinhCity,Vietnam (+84)772954324 darkknightkhtn2008@gmail.com I’maSoftwareEngineerwhohas strongpassionwithinnovation scienceandtechnologies https://www.linkedin.com/in/do-ngoc-tan-64260072/ EXPERIENCE SKILLS Languages:C++20,Rust RustOpenSourceContributor-ForFun: Fluvioisa Golang,Python,Javascript high-performancedistributeddatastreamingplatformthat'swritteninRust, builttomakeiteasytorunreal-timeapplications-https://www.fluvio.io/ MachineLearning: Pytorch May2023-Now ● Contributetofluvioopensource: Blockchain:Tendermint, https://github.com/infinyon/fluvio Cosmos,Ethereum. AWARDS AStartupinDataPrivacy—LeadEngineer(Rust,C++) April2023-Now TopWinnerofBlockchain HackathonEventHostedBy Take theinitiativetore-writesystemfromJStoRust PolicyPal WorldQuant,Vietnam—SeniorEngineer/Lead Engineer(C++):WorldQuant,LLCisaninternationalhedgefundand quantitativeinvestmentmanagementfirmheadquarteredinOldGreenwich, LANGUAGES Connecticut https://www.worldquant.com/ Vietname

In [15]:
document_chunks

[Document(page_content="District7 Do Ngoc Tan HoChiMinhCity,Vietnam (+84)772954324 darkknightkhtn2008@gmail.com I’maSoftwareEngineerwhohas strongpassionwithinnovation scienceandtechnologies https://www.linkedin.com/in/do-ngoc-tan-64260072/ EXPERIENCE SKILLS Languages:C++20,Rust RustOpenSourceContributor-ForFun: Fluvioisa Golang,Python,Javascript high-performancedistributeddatastreamingplatformthat'swritteninRust, builttomakeiteasytorunreal-timeapplications-https://www.fluvio.io/ MachineLearning: Pytorch May2023-Now ● Contributetofluvioopensource: Blockchain:Tendermint, https://github.com/infinyon/fluvio Cosmos,Ethereum. AWARDS AStartupinDataPrivacy—LeadEngineer(Rust,C++) April2023-Now TopWinnerofBlockchain HackathonEventHostedBy Take theinitiativetore-writesystemfromJStoRust PolicyPal WorldQuant,Vietnam—SeniorEngineer/Lead Engineer(C++):WorldQuant,LLCisaninternationalhedgefundand quantitativeinvestmentmanagementfirmheadquarteredinOldGreenwich, LANGUAGES Connecticut https://www.worldqua

In [25]:
raw_pages, metadata = parse_pdf_without_pages(file_path)
print(raw_pages)
print(metadata)


# Step 2.1: Create text chunks
cleaning_functions = [
    merge_hyphenated_words,
    fix_newlines,
    remove_multiple_newlines,
    ]
cleaned_text_pdf = clean_text_without_pages(raw_pages, cleaning_functions)
document_chunks = text_to_docs_without_pages(cleaned_text_pdf, metadata)

["District7\nDo Ngoc Tan\nHoChiMinhCity,Vietnam\n(+84)772954324\ndarkknightkhtn2008@gmail.com\nI’maSoftwareEngineerwhohas strongpassionwithinnovation\nscienceandtechnologies\nhttps://www.linkedin.com/in/do-ngoc-tan-64260072/\nEXPERIENCE SKILLS\nLanguages:C++20,Rust\nRustOpenSourceContributor-ForFun: Fluvioisa\nGolang,Python,Javascript\nhigh-performancedistributeddatastreamingplatformthat'swritteninRust,\nbuilttomakeiteasytorunreal-timeapplications-https://www.fluvio.io/\nMachineLearning:\nPytorch\nMay2023-Now\n● Contributetofluvioopensource: Blockchain:Tendermint,\nhttps://github.com/infinyon/fluvio Cosmos,Ethereum.\nAWARDS\nAStartupinDataPrivacy—LeadEngineer(Rust,C++)\nApril2023-Now TopWinnerofBlockchain\nHackathonEventHostedBy\nTake theinitiativetore-writesystemfromJStoRust\nPolicyPal\nWorldQuant,Vietnam—SeniorEngineer/Lead\nEngineer(C++):WorldQuant,LLCisaninternationalhedgefundand\nquantitativeinvestmentmanagementfirmheadquarteredinOldGreenwich, LANGUAGES\nConnecticut\nhttps://www.w

In [26]:
cleaned_text_pdf

["District7 Do Ngoc Tan HoChiMinhCity,Vietnam (+84)772954324 darkknightkhtn2008@gmail.com I’maSoftwareEngineerwhohas strongpassionwithinnovation scienceandtechnologies https://www.linkedin.com/in/do-ngoc-tan-64260072/ EXPERIENCE SKILLS Languages:C++20,Rust RustOpenSourceContributor-ForFun: Fluvioisa Golang,Python,Javascript high-performancedistributeddatastreamingplatformthat'swritteninRust, builttomakeiteasytorunreal-timeapplications-https://www.fluvio.io/ MachineLearning: Pytorch May2023-Now ● Contributetofluvioopensource: Blockchain:Tendermint, https://github.com/infinyon/fluvio Cosmos,Ethereum. AWARDS AStartupinDataPrivacy—LeadEngineer(Rust,C++) April2023-Now TopWinnerofBlockchain HackathonEventHostedBy Take theinitiativetore-writesystemfromJStoRust PolicyPal WorldQuant,Vietnam—SeniorEngineer/Lead Engineer(C++):WorldQuant,LLCisaninternationalhedgefundand quantitativeinvestmentmanagementfirmheadquarteredinOldGreenwich, LANGUAGES Connecticut https://www.worldquant.com/ Vietnamese,Eng

In [27]:
document_chunks

[Document(page_content="District7 Do Ngoc Tan HoChiMinhCity,Vietnam (+84)772954324 darkknightkhtn2008@gmail.com I’maSoftwareEngineerwhohas strongpassionwithinnovation scienceandtechnologies https://www.linkedin.com/in/do-ngoc-tan-64260072/ EXPERIENCE SKILLS Languages:C++20,Rust RustOpenSourceContributor-ForFun: Fluvioisa Golang,Python,Javascript high-performancedistributeddatastreamingplatformthat'swritteninRust, builttomakeiteasytorunreal-timeapplications-https://www.fluvio.io/ MachineLearning: Pytorch May2023-Now ● Contributetofluvioopensource: Blockchain:Tendermint, https://github.com/infinyon/fluvio Cosmos,Ethereum. AWARDS AStartupinDataPrivacy—LeadEngineer(Rust,C++) April2023-Now TopWinnerofBlockchain HackathonEventHostedBy Take theinitiativetore-writesystemfromJStoRust PolicyPal WorldQuant,Vietnam—SeniorEngineer/Lead Engineer(C++):WorldQuant,LLCisaninternationalhedgefundand quantitativeinvestmentmanagementfirmheadquarteredinOldGreenwich, LANGUAGES Connecticut https://www.worldqua